In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Import libraries:

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

import warnings
warnings.filterwarnings("ignore")

## Overview:

In [ ]:
path = '/kaggle/input/tabular-playground-series-mar-2021/'
os.listdir(path)

In [ ]:
train_data = pd.read_csv(path+'train.csv')
test_data = pd.read_csv(path+'test.csv')
samp = pd.read_csv(path+'sample_submission.csv')

In [ ]:
print(train_data.shape)
train_data.info()
train_data.head()

In [ ]:
print(test_data.shape)
test_data.head()

## EDA and preprocessing:

#### Split features:

In [ ]:
features_cat = ['cat'+str(i) for i in range(19)]
features_num = ['cont'+str(i) for i in range(11)]
no_features = ['id', 'target']

#### Categorical distribution: 

In [ ]:
for i in features_cat:
    sns.displot(train_data, x=i)

#### Numeric distribution

In [ ]:
for i in features_num:
    sns.displot(train_data, x=i)

## Model development:

#### Encoding categorical data:

In [ ]:
le = LabelEncoder()
for col in features_cat:
    le.fit(train_data[col].append(test_data[col]))
    train_data[col] = le.transform(train_data[col])
    test_data[col] = le.transform(test_data[col])

#### Train-Test split:

In [ ]:
X_train = train_data.drop(['id', 'target'], 1)
y_train = train_data['target']
X_test = test_data.drop('id', 1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state=2021)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.fit_transform(X_val)
X_test = sc.fit_transform(X_test)

In [ ]:
model = RandomForestClassifier(n_estimators=200,max_depth=8,random_state=0)

#### Validation:

In [ ]:
model.fit(X_train, y_train)
y_val_pred = model.predict_proba(X_val)
print('Score validation data:', roc_auc_score(y_val, y_val_pred[:, 1]))

In [ ]:
y_pred = model.predict_proba(X_test)

## Submission:

In [ ]:
output = samp.copy()
output['target'] = y_pred[:, 1].round()
output.to_csv('submission.csv', index=False)
output.head()